## Obtendo o conjunto de dados randômico


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
from lsst.rsp import get_tap_service, retrieve_query

#### 1. Definindo parâmetros

In [ ]:
pd.set_option('display.max_rows', 200)#máximo de linhas a serem exibidas
%matplotlib inline
service = get_tap_service("tap")#precisa colocar o 'tap' aqui

In [ ]:
plot_filter_labels = ['u', 'g', 'r', 'i', 'z', 'y']
plot_filter_colors = {'u': '#56b4e9', 'g': '#008060', 'r': '#ff4000',
                      'i': '#850000', 'z': '#6600cc', 'y': '#000000'}
plot_filter_symbols = {'u': 'o', 'g': '^', 'r': 'v', 'i': 's', 'z': '*', 'y': 'p'}

#### 2. Query 

2.1 selecionando objectIds aleatórios(percentual)

In [ ]:
filename = '/scratch/andreiadourado/dp02_dc2_catalogs_Object_all_objectids_sorted.csv'
random.seed(4321)
df = pd.read_csv(filename, 
           skiprows=lambda x: x > 0 and random.random() >=0.006) 

In [ ]:
len(df)

Devido ao limite de dados que podem ser obtidos na estrutura do RSP, só foi possível obter um máximo de 50000 galáxias por vez, sendo necessário repetir os passos seguintes quantas vezes forem necessárias até obter a quantidade de dados desejada

2.2 escrevendo os objectIds no formato da query

In [ ]:
ids = np.squeeze((df.values)[0:50000])
len(ids)

In [ ]:
obj_ids = str(tuple(ids))

2.3 escrevendo a query (com o JOIN entre a tabela Object e a tabela TruthSummary)

In [ ]:
%%time
query = '\n'.join(['SELECT mt.id_truth_type, mt.match_objectId, obj.objectId, obj.detect_isPrimary, ts.ra, ts.dec, ts.redshift, scisql_nanojanskyToAbMag(ts.flux_u_noMW) as u_Mag, scisql_nanojanskyToAbMag(ts.flux_g_noMW) as g_Mag, scisql_nanojanskyToAbMag(ts.flux_r_noMW) as r_Mag, scisql_nanojanskyToAbMag(ts.flux_i_noMW) as i_Mag, scisql_nanojanskyToAbMag(ts.flux_z_noMW) as z_Mag, scisql_nanojanskyToAbMag(ts.flux_y_noMW) as y_Mag ',
        'FROM dp02_dc2_catalogs.MatchesTruth AS mt ',
        'JOIN dp02_dc2_catalogs.TruthSummary AS ts ',
        'ON mt.id_truth_type=ts.id_truth_type ',
        'JOIN dp02_dc2_catalogs.Object AS obj ',
        'ON mt.match_objectId=obj.objectId ', 
        'WHERE obj.objectId IN'+obj_ids,
        'AND ts.truth_type=1 ',
        'AND obj.detect_isPrimary=1'])

print(query)
#print(' ')
#results = service.search(query1).to_table().to_pandas()#converte para tabela astropy e depois pra pandas dataframe
#print('len(results) = ', len(results))

Sugestão da Melissa no fórum:

In [ ]:
job1 = service.submit_job(query1)
print('Job URL is', job1.url)
print('Job phase is', job1.phase)

In [ ]:
%%time
job1.run()

In [ ]:
print('Job phase is now ', job1.phase)

In [ ]:
async_results1 = job1.fetch_result().to_table().to_pandas()

In [ ]:
len(async_results1)

In [ ]:
async_results1.to_csv("DadosRandom17.csv")

In [ ]:
plt.scatter(async_results1['ra'],async_results1['dec'], c=async_results1['redshift'],cmap='turbo',s=1)
plt.colorbar()